# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [142]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [143]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [144]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        #print(row[0], '|', row[2])
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [145]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [146]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [147]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [148]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

To answer this query, we need to select artist, song, and song length from the table and then filter with sessionId and itemInSession.

The CQL statement will be: SELECT artist, song, song_length FROM session_songs WHERE sessionId = 338 and itemInSession = 4

Considerations for data modeling:
    
    - The primary key will consist of partition key sessionId, and the clustering key itemInSession because we need to filter by those fields.    
    - The table name will be: session_songs
    - The columns in the table are: sessionId, itemInSession, artist, song, song_length

In [149]:
# Create table for query 1
query = "CREATE TABLE IF NOT EXISTS session_songs"
query = query + "(sessionId int, itemInSession int, artist text, song text, song_length decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [150]:
# Insert data from event_datafile_new.csv to the table for answering query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

#### Do a SELECT to verify that the data have been inserted into each table

In [151]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, song_length FROM session_songs WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2:
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this query, we need to select itemInSession, artist, song, user from the table and then filter with userId and sessionId.

The CQL statement will be: SELECT itemInSession, artist, song, user FROM user_songs WHERE sessionId = 338 and itemInSession = 4

Considerations for data modeling:
    
    - The primary key will consist of composite partition key (userId, SessionId). The reason is that we want to create a unique combination of userId and sessionId for each node. If we just use userId as the partition key, the sessionId will be duplicated for the same userId in each node.
    - The clustering key will be itemInSession because we need to sort by this field.
    - The table name will be: user_songs
    - The columns in the table are: userId, sessionId, itemInSession, artist, song, user

In [152]:
# Create table for query 2
query = "CREATE TABLE IF NOT EXISTS user_songs"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, user text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [153]:
# Insert data from event_datafile_new.csv to the table for answering query 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

In [154]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT itemInSession, artist, song, user FROM user_songs WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.user)                    

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3:
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this query, we need to select user from the table and then filter with song.

The CQL statement will be: SELECT user FROM app_history WHERE song = 'All Hands Against His Own'

Considerations for data modeling:
    
    - The primary key will consist of partition key song and the clustering column will be userId. This is because we need to filter by the song title and then sort each user by the user_id.
    - The table name will be: app_history
    - The columns in the table are: userId, user, song

In [155]:
# Create table for query 3
query = "CREATE TABLE IF NOT EXISTS app_history"
query = query + "(userId int, user text, song text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [156]:
# Insert data from event_datafile_new.csv to the table for answering query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO app_history (userId, user, song)"
        query = query + "VALUES (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), line[1] + " " + line[4], line[9]))

In [157]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT user FROM app_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)     

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [158]:
## Drop the table before closing out the sessions

In [159]:
query = "drop table session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [160]:
query = "drop table user_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [161]:
query = "drop table app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [162]:
session.shutdown()
cluster.shutdown()